# Library of useful functions

In [5]:
# Primes
using Primes

# ToDo: Generalize this so that it can take pregenerated sieves as input.
function primes(limit::Integer)::Array{Integer, 1}
	sieve = Integer[2]

	for odd ∈ 3:2:limit    # Consider only odd numbers
		isprime = true
		for prime ∈ sieve
			prime * prime > odd && break    # No need to check beyond this
			odd % prime == 0 && (isprime = false)
		end
		isprime && push!(sieve, odd)
	end

	return sieve
end

isprime(n::Integer) = (primes(n)[end] == n)

function nth_prime(n::Integer)::Integer
	upperbound = Integer(floor(n * (log(n) + log(log(n)))))
	return primes(upperbound)[n]
end


# Factorization
function prime_factorize(n::Integer)
	sieve = primes(n)    # Have to figure out a way to not generate so many primes

	factors = Integer[]
	powers = Integer[]

	for prime ∈ sieve
		prime > n && break
		pow = 0
		while n % prime == 0
			n ÷= prime
			pow += 1
		end
		if pow > 0
			push!(factors, prime)
			push!(powers, pow)
		end
	end

	return collect(zip(factors, powers))
end

function count_factors(n::Integer)::Integer
	n == 1 && return 1
	factorization = prime_factorize(n)
	count = 1
	return prod(map(x -> x[2] + 1, factorization))
end


# The following are implemented in Julia.Base
# # The ith position gives the coefficient of the (i-1)th power of the base.
# function digits(n::Integer, base::Integer)::Array{Integer, 1}
# 	d = Integer[]
# 	while n > 0
# 		push!(d, n % base)
# 		n ÷= base
# 	end
# 	return d
# end

# # Combinatorics
# function binomial(n::Integer, k::Integer)::Integer
# 	product = 1
# 	for i ∈ 0:(k - 1)
# 		product *= (n - i) / (k - i)
# 	end
# 	return Integer(product)
# end

# count_digits(n::Integer, base::Integer) = length(digits(n, base))

count_factors (generic function with 1 method)

# Problem 001: Multiples of 3 and 5

If we list all the natural numbers below $ 10 $ that are multiples of $ 3 $ or $ 5 $, we get $ 3 $, $ 5 $, $ 6 $ and $ 9 $. The sum of these multiples is $ 23 $.

Find the sum of all the multiples of $ 3 $ or $ 5 $ below $ 1000 $.

In [3]:
function multiples(limit::Integer)::Integer
	sum::Integer = 0
	for i ∈ 3:limit
		if i%3 == 0 || i%5 == 0
			sum += i
		end
	end
	return sum
end
println(multiples(999))

233168


# Problem 002: Even Fibonacci numbers

Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with $ 1 $ and $ 2 $, the first 10 terms will be
$ 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, … $.

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

In [4]:
function even_Fibonacci(stop::Integer)::Integer
	sum::Integer = 10
	a::Integer = 2
	b::Integer = 8
	while 4b + a < stop
		(a, b) = (b, 4b + a)
		sum += b
	end
	return sum
end
println(even_Fibonacci(4000000))

4613732


# Problem 003: Largest prime factor

The prime factors of $ 13195 $ are $ 5, 7, 13, 29 $.

What is the largest prime factor of the number $ 600851475143 $?

In [5]:
# The idea is to keep dividing by the small divisors so that we are left with the largest prime number.
function largest_prime_factor(n::Integer)::Integer
	# Special case: 2^k is a factor
	while n ≠ 2 && n % 2 == 0
		n ÷= 2
	end

	# Odd prime factors p^k
	i::Integer = 3
	while i * i ≤ n
		if isprime(i) && n % i == 0
			n ÷= i
		else
			i += 2
		end
	end

	return n
end

println(largest_prime_factor(600851475143))

6857


# Problem 004: Largest palindrome product

A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is $ 9009 = 91 × 99 $.

Find the largest palindrome made from the product of two 3-digit numbers.

Note that $ 999 ⋅ 998 = 998 ⋅ 999 $, so we only need to check the upper diagonal matrix.

In [6]:
function reverse(n::Integer)::Integer
	decimal = digits(n)
	rev::Integer = 0
	for d ∈ decimal
		rev = 10 * rev + d
	end
	return rev
end

ispalindrome(n::Integer) = (n == reverse(n))

function largest_palindrome_product(max_decimal_digits::Integer)::Integer
	max::Integer = 10^max_decimal_digits - 1
	palindromes = Integer[]

	for i ∈ Iterators.reverse(1:max), j ∈ Iterators.reverse(i:max)
		n::Integer = i * j

		# The i*max cannot go below the any of the palindromes found.
		length(palindromes) > 0 && i * max ≤ palindromes[1] && break

		ispalindrome(n) && push!(palindromes, n)
	end
	return maximum(palindromes)
end

println(largest_palindrome_product(3))

906609


# Problem 005: Smallest multiple

$ 2520 $ is the smallest number that can be divided by each of the numbers from $ 1 $ to $ 10 $ without any remainder.

What is the smallest positive number that is evenly divisible by all of the numbers from $ 1 $ to $ 20 $?

In [6]:
2^4 * 3^2 * 5 * 7 * 11 * 13 * 17 * 19

232792560

# Problem 006: Sum square difference

The sum of the squares of the first ten natural numbers is
$ 1^2 + 2^2 + ⋯ + 10^2 = 385 $.

The square of the sum of the first ten natural numbers is
$ (1 + 2 + ⋯ + 10)^2 = 55^2 = 3025 $.

Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is $ 3025 - 385 = 2640 $.

Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

$$ Δ = \left( \frac{n (n+1)}{2} \right)^2 - \frac{n (n + 1) (2n + 1)}{6} = \frac{1}{12} (n (n + 1) (3 n^2 - n - 2)) $$

In [7]:
100 * (100 + 1) * (3 * 100^2 - 100 - 2) ÷ 12

25164150

# Problem 007: 10001st prime

By listing the first six prime numbers: $ 2, 3, 5, 7, 11, 13 $, we can see that the 6th prime is $ 13 $.

What is the 10001st prime number?

The trick is to use upper bound for the nth prime as given [here](https://en.m.wikipedia.org/wiki/Prime_number_theorem#Approximations_for_the_nth_prime_number).

In [6]:
println(nth_prime(10001))

104743


# Problem 008: Largest product in a series

The four adjacent digits in the 1000-digit number that have the greatest product are $ 9 × 9 × 8 × 9 = 5832 $.

```
73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450
```

Find the thirteen adjacent digits in the 1000-digit number that have the greatest product. What is the value of this product?

In [8]:
n = 7316717653133062491922511967442657474235534919493496983520312774506326239578318016984801869478851843858615607891129494954595017379583319528532088055111254069874715852386305071569329096329522744304355766896648950445244523161731856403098711121722383113622298934233803081353362766142828064444866452387493035890729629049156044077239071381051585930796086670172427121883998797908792274921901699720888093776657273330010533678812202354218097512545405947522435258490771167055601360483958644670632441572215539753697817977846174064955149290862569321978468622482839722413756570560574902614079729686524145351004748216637048440319989000889524345065854122758866688116427171479924442928230863465674813919123162824586178664583591245665294765456828489128831426076900422421902267105562632111110937054421750694165896040807198403850962455444362981230987879927244284909188845801561660979191338754992005240636899125607176060588611646710940507754100225698315520005593572972571636269561882670428252483600823257530420752963450

function largest_product(n::BigInt, adjacent_digits::Integer)::Integer
	decimal = digits(n)
	product_max::Integer = 1 

	for i ∈ 1:(length(decimal) - adjacent_digits + 1)
		product::Integer = 1
		for k ∈ 0:(adjacent_digits - 1)
			product *= decimal[i + k]
		end
		product > product_max && (product_max = product)
		end
	end

	return product_max
end

println(largest_product(n, 13))

23514624000


# Problem 009: Special Pythagorean triplet

A Pythagorean triplet is a set of three natural numbers, $ a < b < c $, for which, $ a^2 + b^2 = c^2 $.
For example, $ 3^2 + 4^2 = 9 + 16 = 25 = 5^2 $.

There exists exactly one Pythagorean triplet for which $ a + b + c = 1000 $.
Find the product abc.

# Problem 010: Summation of primes

The sum of the primes below $ 10 $ is $ 2 + 3 + 5 + 7 = 17 $.

Find the sum of all the primes below two million.

In [9]:
println(sum(primes(2000000)))

142913828922


# Problem 011: Largest product in a grid

In the 20×20 grid below, four numbers along a diagonal line have been marked in bold.
<pre>
08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 <b>26</b> 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 <b>63</b> 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 <b>78</b> 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 <b>14</b> 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48
</pre>

The product of these numbers is $ 26 × 63 × 78 × 14 = 1788696 $.

What is the greatest product of four adjacent numbers in the same direction (up, down, left, right, or diagonally) in the 20×20 grid?

The code below works, but does not show in the Jupyter Lab viewer. I am sure it has something to do with the leading zeros in the numbers. I do not know why they would matter, though. One can always go to the JSON view and read the code in `cell[22]`.

In [10]:
numbers = Integer[
	08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08;
	49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00;
	81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65;
	52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91;
	22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80;
	24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50;
	32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70;
	67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21;
	24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72;
	21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95;
	78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92;
	16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57;
	86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58;
	19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40;
	04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66;
	88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69;
	04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36;
	20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16;
	20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54;
	01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48;
]

function largestproduct(numbers::Array{Integer, 2}, moves::Integer)::Integer
	# ToDo: Update everything to allow for custom indexing
	# https://docs.julialang.org/en/v1/devdocs/offset-arrays/
	# https://julialang.org/blog/2016/02/iteration/
	# https://docs.julialang.org/en/v1/manual/arrays/

	dim = size(numbers)
	productmax = 1

	# Along dimension 1
	for i ∈ 1:dim[1], j ∈ 1:(dim[2] - moves)
		product = 1
		for k ∈ 0:(moves-1)
			product *= numbers[i, j + k]
		end
		if product > productmax
			productmax = product
		end
	end

	# Along dimension 2
	for j ∈ 1:dim[2], i ∈ 1:(dim[1] - moves)
		product = 1
		for k ∈ 0:(moves-1)
			product *= numbers[i + k, j]
		end
		if product > productmax
			productmax = product
		end
	end

	# Diagonal
	for i ∈ 1:dim[1], j ∈ 1:dim[2]
		product = 1
		for k ∈ 0:(moves-1)
			if (i + k > dim[1]) || (j + k > dim[2])
				break
			end
			product *= numbers[i + k, j + k]
		end
		if product > productmax
			productmax = product
		end
	end

	# Antidiagonal
	for i ∈ 1:dim[1], j ∈ 1:dim[2]
		product = 1
		for k ∈ 0:(moves-1)
			if (i + k > dim[1]) || (j - k < 1)
				break
			end
			product *= numbers[i + k, j - k]
		end
		if product > productmax
			productmax = product
		end
	end

	return productmax
end

println(largestproduct(numbers, 4))

70600674


# Problem 012: Highly divisible triangular number

The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be $ 1 + 2 + 3 + 4 + 5 + 6 + 7 = 28 $.

The first ten terms would be
$ 1, 3, 6, 10, 15, 21, 28, 36, 45, 55, … $.

Let us list the factors of the first seven triangle numbers:

```
 1: 1
 3: 1,3
 6: 1,2,3,6
10: 1,2,5,10
15: 1,3,5,15
21: 1,3,7,21
28: 1,2,4,7,14,28
```
We can see that $ 28 $ is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred divisors?

In [ ]:
function first_triangular_number(factors::Integer)::Integer
	s::Integer = 0
	for i ∈ Iterators.countfrom(1)
		s += i
		count_factors(s) > factors && return s
	end
end

first_triangular_number(500)

# Problem 013: Large sum

Work out the first ten digits of the sum of the following one-hundred 50-digit numbers.

In [2]:
numbers = BigInt[
	37107287533902102798797998220837590246510135740250,
	46376937677490009712648124896970078050417018260538,
	74324986199524741059474233309513058123726617309629,
	91942213363574161572522430563301811072406154908250,
	23067588207539346171171980310421047513778063246676,
	89261670696623633820136378418383684178734361726757,
	28112879812849979408065481931592621691275889832738,
	44274228917432520321923589422876796487670272189318,
	47451445736001306439091167216856844588711603153276,
	70386486105843025439939619828917593665686757934951,
	62176457141856560629502157223196586755079324193331,
	64906352462741904929101432445813822663347944758178,
	92575867718337217661963751590579239728245598838407,
	58203565325359399008402633568948830189458628227828,
	80181199384826282014278194139940567587151170094390,
	35398664372827112653829987240784473053190104293586,
	86515506006295864861532075273371959191420517255829,
	71693888707715466499115593487603532921714970056938,
	54370070576826684624621495650076471787294438377604,
	53282654108756828443191190634694037855217779295145,
	36123272525000296071075082563815656710885258350721,
	45876576172410976447339110607218265236877223636045,
	17423706905851860660448207621209813287860733969412,
	81142660418086830619328460811191061556940512689692,
	51934325451728388641918047049293215058642563049483,
	62467221648435076201727918039944693004732956340691,
	15732444386908125794514089057706229429197107928209,
	55037687525678773091862540744969844508330393682126,
	18336384825330154686196124348767681297534375946515,
	80386287592878490201521685554828717201219257766954,
	78182833757993103614740356856449095527097864797581,
	16726320100436897842553539920931837441497806860984,
	48403098129077791799088218795327364475675590848030,
	87086987551392711854517078544161852424320693150332,
	59959406895756536782107074926966537676326235447210,
	69793950679652694742597709739166693763042633987085,
	41052684708299085211399427365734116182760315001271,
	65378607361501080857009149939512557028198746004375,
	35829035317434717326932123578154982629742552737307,
	94953759765105305946966067683156574377167401875275,
	88902802571733229619176668713819931811048770190271,
	25267680276078003013678680992525463401061632866526,
	36270218540497705585629946580636237993140746255962,
	24074486908231174977792365466257246923322810917141,
	91430288197103288597806669760892938638285025333403,
	34413065578016127815921815005561868836468420090470,
	23053081172816430487623791969842487255036638784583,
	11487696932154902810424020138335124462181441773470,
	63783299490636259666498587618221225225512486764533,
	67720186971698544312419572409913959008952310058822,
	95548255300263520781532296796249481641953868218774,
	76085327132285723110424803456124867697064507995236,
	37774242535411291684276865538926205024910326572967,
	23701913275725675285653248258265463092207058596522,
	29798860272258331913126375147341994889534765745501,
	18495701454879288984856827726077713721403798879715,
	38298203783031473527721580348144513491373226651381,
	34829543829199918180278916522431027392251122869539,
	40957953066405232632538044100059654939159879593635,
	29746152185502371307642255121183693803580388584903,
	41698116222072977186158236678424689157993532961922,
	62467957194401269043877107275048102390895523597457,
	23189706772547915061505504953922979530901129967519,
	86188088225875314529584099251203829009407770775672,
	11306739708304724483816533873502340845647058077308,
	82959174767140363198008187129011875491310547126581,
	97623331044818386269515456334926366572897563400500,
	42846280183517070527831839425882145521227251250327,
	55121603546981200581762165212827652751691296897789,
	32238195734329339946437501907836945765883352399886,
	75506164965184775180738168837861091527357929701337,
	62177842752192623401942399639168044983993173312731,
	32924185707147349566916674687634660915035914677504,
	99518671430235219628894890102423325116913619626622,
	73267460800591547471830798392868535206946944540724,
	76841822524674417161514036427982273348055556214818,
	97142617910342598647204516893989422179826088076852,
	87783646182799346313767754307809363333018982642090,
	10848802521674670883215120185883543223812876952786,
	71329612474782464538636993009049310363619763878039,
	62184073572399794223406235393808339651327408011116,
	66627891981488087797941876876144230030984490851411,
	60661826293682836764744779239180335110989069790714,
	85786944089552990653640447425576083659976645795096,
	66024396409905389607120198219976047599490197230297,
	64913982680032973156037120041377903785566085089252,
	16730939319872750275468906903707539413042652315011,
	94809377245048795150954100921645863754710598436791,
	78639167021187492431995700641917969777599028300699,
	15368713711936614952811305876380278410754449733078,
	40789923115535562561142322423255033685442488917353,
	44889911501440648020369068063960672322193204149535,
	41503128880339536053299340368006977710650566631954,
	81234880673210146739058568557934581403627822703280,
	82616570773948327592232845941706525094512325230608,
	22918802058777319719839450180888072429661980811197,
	77158542502016545090413245809786882778948721859617,
	72107838435069186155435662884062257473692284509516,
	20849603980134001723930671666823555245252804609722,
	53503534226472524250874054075591789781264330331690,
]

function large_sum(numbers::Array{BigInt, 1})::Integer
	s::BigInt = sum(numbers)
	numdigits = length(digits(s))
	strip = numdigits - 10
	return s ÷ BigInt(10)^strip
end

println(large_sum(numbers))

5537376230


# Problem 014: Longest Collatz sequence

The following iterative sequence is defined for the set of positive integers:

$ n → n/2 $ ($ n $ is even)

$ n → 3n + 1 $ ($ n $ is odd)

Using the rule above and starting with $ 13 $, we generate the following sequence:

$ 13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1 $.

It can be seen that this sequence (starting at $ 13 $ and finishing at $ 1 $) contains $ 10 $ terms. Although it has not been proved yet (Collatz Problem), it is thought that all starting numbers finish at $ 1 $.

Which starting number, under one million, produces the longest chain?

**NOTE**: Once the chain starts the terms are allowed to go above one million.

In [5]:
Collatz_transform(n::Int)::Int  =  iseven(n) ? n ÷ 2 : 3n + 1

Collatz_length(start::Int)::Int  =  start == 1 ? 1 : 1 + Collatz_length(Collatz_transform(start))

println(argmax(map(Collatz_length, 1:1_000_000)))

# This code may be optimized by memoization as a whole. I am sure a lot of paths are repeated.

837799


# Problem 015: Lattice paths

Starting in the top left corner of a 2×2 grid, and only being able to move to the right and down, there are exactly 6 routes to the bottom right corner.

How many such routes are there through a 20×20 grid?

In [10]:
function lattice_paths(size::Integer)::Integer
	return binomial(2*size, size)
end

println(lattice_paths(20))

137846528820


# Problem 016: Power digit sum

$ 2^{15} = 32768 $ and the sum of its digits is $ 3 + 2 + 7 + 6 + 8 = 26 $.

What is the sum of the digits of the number $ 2^{1000} $?

In [1]:
sum(digits(BigInt(2)^1000))

1366

# Problem 017: Number letter counts

If the numbers $ 1 $ to $ 5 $ are written out in words: one, two, three, four, five, then there are $ 3 + 3 + 5 + 4 + 4 = 19 $ letters used in total.

If all the numbers from $ 1 $ to $ 1000 $ (one thousand) inclusive were written out in words, how many letters would be used?

*NOTE*: Do not count spaces or hyphens. For example, 342 (three hundred and forty-two) contains 23 letters and 115 (one hundred and fifteen) contains 20 letters. The use of "and" when writing out numbers is in compliance with British usage.

In [2]:
"""This uses short scales.
https://en.wikipedia.org/wiki/Long_and_short_scales"""
function decimaltotext(n::Int)::String
	# We do not need momoization beyond 999 because things become regular.
	store = Array{String, 1}(undef, 999)
	store[1:19] = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen"]

	function decimaltotext_aux(n::Int, memo::Array{String, 1})::String
		tens = ["ten", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

		if isassigned(memo, n)
			return memo[n]
		end

		# Put spaces between words to make this more legible.
		if 20 ≤ n ≤ 99
			if n % 10 == 0
				return tens[n ÷ 10]
			else
				return string(tens[n ÷ 10], decimaltotext_aux(n % 10, memo))
			end
		elseif 100 ≤ n ≤ 999
			if n % 100 == 0
				return string(decimaltotext_aux(n ÷ 100, memo), "hundred")
			else
				return string(decimaltotext_aux(n ÷ 100, memo), "hundredand", decimaltotext_aux(n % 100, memo))
			end
		elseif 1_000 ≤ n ≤ 999_999
			if n % 1_000 == 0
				return string(decimaltotext_aux(n ÷ 1_000, memo), "thousand")
			else
				return string(decimaltotext_aux(n ÷ 1_000, memo), "thousandand", decimaltotext_aux(n % 1_000, memo))
			end
		elseif 1_000_000 ≤ n ≤ 999_999_999
			if n % 1_000_000 == 0
				return string(decimaltotext_aux(n ÷ 1_000_000, memo), "million")
			else
				return string(decimaltotext_aux(n ÷ 1_000_000, memo), "millionand", decimaltotext_aux(n % 1_000_000, memo))
			end
		elseif 1_000_000_000 ≤ n ≤ 999_999_999_999
			if n % 1_000_000_000 == 0
				return string(decimaltotext_aux(n ÷ 1_000_000_000, memo), "billion")
			else
				return string(decimaltotext_aux(n ÷ 1_000_000_000, memo), "billionand", decimaltotext_aux(n % 1_000_000_000, memo))
			end
		elseif 1_000_000_000_000 ≤ n
			if n % 1_000_000_000_000 == 0
				return string(decimaltotext_aux(n ÷ 1_000_000_000_000, memo), "trillion")
			else
				return string(decimaltotext_aux(n ÷ 1_000_000_000_000, memo), "trillionand", decimaltotext_aux(n % 1_000_000_000_000, memo))
			end
		end
	end

	return decimaltotext_aux(n, store)
end

println(sum(map(length ∘ decimaltotext, 1:1000)))

21124


# Problem 018: Maximum path sum I

By starting at the top of the triangle below and moving to adjacent numbers on the row below, the maximum total from top to bottom is $ 23 $.
```
3
7 4
2 4 6
8 5 9 3
```
That is, $ 3 + 7 + 4 + 9 = 23 $.

Find the maximum total from top to bottom of the triangle below:
```
75
95 64
17 47 82
18 35 87 10
20 04 82 47 65
19 01 23 75 03 34
88 02 77 73 07 63 67
99 65 04 28 06 16 70 92
41 41 26 56 83 40 80 70 33
41 48 72 33 47 32 37 16 94 29
53 71 44 65 25 43 91 52 97 51 14
70 11 33 28 77 73 17 78 39 68 17 57
91 71 52 38 17 14 91 43 58 50 27 29 48
63 66 04 68 89 53 67 30 73 16 69 87 40 31
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23
```
For a scaled up version, see **Problem 67**.

In [5]:
triangle =
"75
95 64
17 47 82
18 35 87 10
20 04 82 47 65
19 01 23 75 03 34
88 02 77 73 07 63 67
99 65 04 28 06 16 70 92
41 41 26 56 83 40 80 70 33
41 48 72 33 47 32 37 16 94 29
53 71 44 65 25 43 91 52 97 51 14
70 11 33 28 77 73 17 78 39 68 17 57
91 71 52 38 17 14 91 43 58 50 27 29 48
63 66 04 68 89 53 67 30 73 16 69 87 40 31
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23"

function numberarray(input::String)::Array{Int, 2}
	lines = map(split, split(input, "\n"))
	lines = filter(x -> length(x) > 0, lines)    #	Remove empty lines
	# Put all this in an T-array
	n = length(lines)
	Δ = zeros(Int, (n, n))
	for i ∈ 1:n, j ∈ 1:i
		Δ[i, j] = parse(Int, lines[i][j])
	end
	return Δ
end

function max_path_sum(triangle::Array{Int, 2})::Int
	store = zeros(Int, size(triangle))
	store[end, :] = triangle[end, :]    # Base case

	function max_path_sum_aux(triangle::Array{Int, 2}, memo::Array{Int, 2}, row::Int, col::Int)::Int
		memo[row, col] > 0 && (return memo[row, col])
		memo[row, col] = triangle[row, col] + max(max_path_sum_aux(triangle, memo, row + 1, col), max_path_sum_aux(triangle, memo, row + 1, col + 1))
		return memo[row, col]
	end
	return max_path_sum_aux(triangle, store, 1, 1)
end

println(max_path_sum(numberarray(triangle)))

1074


# Problem 20: Factorial digit sum

$ n! $ means $ n × (n − 1) × ... × 3 × 2 × 1 $.

For example, $ 10! = 10 × 9 × ... × 3 × 2 × 1 = 3628800 $,
and the sum of the digits in the number $ 10! $ is $ 3 + 6 + 2 + 8 + 8 + 0 + 0 = 27 $.

Find the sum of the digits in the number $ 100! $.

In [4]:
sum(digits(factorial(big(100))))

648

# Problem 22: Names scores

Using `names.txt` (right click and 'Save Link/Target As...'), a 46K text file containing over five-thousand first names, begin by sorting it into alphabetical order. Then working out the alphabetical value for each name, multiply this value by its alphabetical position in the list to obtain a name score.

For example, when the list is sorted into alphabetical order, COLIN, which is worth $ 3 + 15 + 12 + 9 + 14 = 53 $, is the 938th name in the list. So, COLIN would obtain a score of $ 938 × 53 = 49714 $.

What is the total of all the name scores in the file?

In [7]:
alphabeticvalue(name::String)::Int  =  sum(char - 'A' + 1 for char in name)

function getwords(path::AbstractString)::Array{String, 1}
	words = open(path) do file
	    read(file, String)
	end
	return map(string, split(words, r"\W", keepempty=false))    # SubString to String
end

function totalscore(words::Array{String, 1})::Int
	sorted = sort(words)
	total::Int = 0
	for i ∈ 1:length(sorted)
		total += alphabeticvalue(sorted[i]) * i
	end
	return total
end

println(totalscore(getwords("p022_names.txt")))

871198282


# Problem 028: Number spiral diagonals

Starting with the number $ 1 $ and moving to the right in a clockwise direction a 5 by 5 spiral is formed as follows:
```
21 22 23 24 25
20  7  8  9 10
19  6  1  2 11
18  5  4  3 12
17 16 15 14 13
```
It can be verified that the sum of the numbers on the diagonals is $ 101 $.

What is the sum of the numbers on the diagonals in a 1001 by 1001 spiral formed in the same way?

The matrix can be written as
$$ \begin{matrix}
3^2 + 2 ⋅ 3 ⋅ 3 &  &  &  &  &  & 3^2 + 2 ⋅ 3 ⋅ 4 \\ 
 & 3^2 + 2 ⋅ 2 ⋅ 3 &  &  &  & 3^2 + 2 ⋅ 2 ⋅ 4 & \\ 
 &  & 1^2 + 2 ⋅ 1 ⋅ 3 &  & 1^2 + 2 ⋅ 1 ⋅ 4 &  & \\ 
 &  &  & 1 &  &  & \\ 
 &  & 1^2 + 2 ⋅ 1 ⋅ 2 &  & 1^2 + 2 ⋅ 1 ⋅ 1 &  & \\ 
 & 3^2 + 2 ⋅ 2 ⋅ 2 &  &  &  & 3^2 + 2 ⋅ 2 ⋅ 1 & \\ 
3^2 + 2 ⋅ 3 ⋅ 2 &  &  &  &  &  & 3^2 + 2 ⋅ 3 ⋅ 1
\end{matrix} $$

Let the dimension of the matrix be $ n \times n $, where $ n $ is necessarily odd. Let $ m = \frac{n - 1}{2} $. Then, going from inside to outside, the required sum is
$$ 1 + \sum_{i = 1}^m \left[ 4(-1 + 2i)^2 + 2 i (1 + 2 + 3 + 4) \right] = 1 + 4m + 2m(m+1) + \frac83 m(m+1)(2m+1) . $$

In [6]:
function number_spiral_diagonals(n::Integer)    # n must be odd
	m::Integer = (n - 1) ÷ 2
	return 1 + 4 * m + 2 * m * (m+1) + 8 * m * (m+1) * (2m+1) ÷ 3
end

println(number_spiral_diagonals(1001))

669171001


# Problem 34: Digit factorials

$ 145 $ is a curious number, as $ 1! + 4! + 5! = 1 + 24 + 120 = 145 $.

Find the sum of all numbers which are equal to the sum of the factorial of their digits.

Note: as $ 1! = 1 $ and $ 2! = 2 $ are not sums they are not included.

# Problem 035: Circular primes

The number $ 197 $, is called a circular prime because all rotations of the digits: 197, 971, and 719, are themselves prime.

There are thirteen such primes below $ 100 $: $ 2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79, 97 $.

How many circular primes are there below one million?

In [12]:
# This code is also suboptimal. It checks the primality of all of the rotations.
# For example, it will check if 197 is prime while checking for the prime circularity of each of 197, 971, and 719.

function rotate(n::Integer)::Array{Integer, 1}
	numdigits::Integer = length(digits(n))
	rotations = Array{Integer, 1}(undef, numdigits)
	rotations[1] = n
	for i ∈ 1:(numdigits - 1)
		rotated = (rotations[i] % 10) * 10^(numdigits-1) + (rotations[i] ÷ 10)
		rotations[i + 1] = rotated
	end
	return rotations
end

function count_circular_primes(limit::Integer)::Integer
	count::Integer = 0
	sieve = primes(limit)
	for p ∈ sieve
		rotations = rotate(p)
		all(map(x -> x ∈ sieve, rotations)) && (count += 1)
	end
	return count
end

println(count_circular_primes(1000000))

InterruptException: InterruptException:

# Problem 041: Pandigital prime

We shall say that an $ n $-digit number is pandigital if it makes use of all the digits $ 1 $ to $ n $ exactly once. For example, $ 2143 $ is a $ 4 $-digit pandigital and is also prime.

What is the largest $ n $-digit pandigital prime that exists?

Clearly, for a $ n $-digit pandigital prime, it must be less than $ 987654321$. But the code does not run in finite time. **ToDo**.

In [ ]:
function ispandigital(n::Integer)::Bool
	decimal = digits(n)

	for i ∈ 1:length(decimal)
		length(filter(x -> x == i, decimal)) ≠ 1 && return false
	end

	for i ∈ union([0], (length(decimal)+1):9)
		length(filter(x -> x == i, decimal)) ≠ 0 && return false
	end

	return true
end

function largest_pandigital_prime()::Integer
	sieve = primes(987654321)
	for prime ∈ Iterators.reverse(sieve)
		ispandigital(prime) && return prime
	end
end

println(largest_pandigital_prime())

In [15]:
length(primes(100))

25

# Problem 067: Maximum path sum II

By starting at the top of the triangle below and moving to adjacent numbers on the row below, the maximum total from top to bottom is 23.
```
3
7 4
2 4 6
8 5 9 3
```
That is, $ 3 + 7 + 4 + 9 = 23 $.

Find the maximum total from top to bottom in `p067_triangle.txt` (right click and 'Save Link/Target As...'), a 15K text file containing a triangle with one-hundred rows.

Comment: For the code, see **Problem 18**.

In [6]:
triangle = open("p067_triangle.txt") do file
    read(file, String)
end
println(max_path_sum(numberarray(triangle)))

7273
